In [ ]:
from pycalphad import Database, variables as v
from localeq import composition_set, local_equilibrium
import numpy as np
from collections import defaultdict

In [ ]:
dbf = Database('databases/mc_fe_v2.059.pycalphad.tdb')

# T0 Computation
The T0 (or T-zero) temperature is a thermodynamic quantity defined by an equality of the molar Gibbs energies of two different phases. T0 is relevant to the design of heat treatments for some alloys because, below the T0 temperature, so-called 'massive' transformation kinetics become active and may dominate the observed evolution of the microstructure. Massive transformations are diffusionless and can occur very rapidly, with no transformation barrier.

In [ ]:
comps = ['FE', 'MN', 'VA']

state_variables = {v.N: 1, v.P: 1e5, v.T: 300}

fcc_composition_sets = [
                    composition_set(dbf, comps, 'FCC_A1',
                                    {**state_variables},
                                    phase_amt=1,
                                   ),  
                   ]
bcc_composition_sets = [
                    composition_set(dbf, comps, 'BCC_A2',
                                    {**state_variables},
                                    phase_amt=1,
                                   ),  
                   ]
x = []
y = defaultdict(lambda: [])
for temperature in np.arange(300., 2000.):
    state_variables[v.T] = temperature
    for compset in fcc_composition_sets:
        compset.dof[2] = temperature
        compset.update(compset.dof[2:], compset.NP, compset.dof[:2])
    for compset in bcc_composition_sets:
        compset.dof[2] = temperature
        compset.update(compset.dof[2:], compset.NP, compset.dof[:2])
    result_fcc, composition_sets = local_equilibrium(fcc_composition_sets, comps, {**state_variables, v.X('MN'): 0.1})
    result_bcc, composition_sets = local_equilibrium(bcc_composition_sets, comps, {**state_variables, v.X('MN'): 0.1})

    if not result_fcc.converged:
        print(temperature)
        raise ValueError('Convergence failure')
    if not result_bcc.converged:
        print(temperature)
        raise ValueError('Convergence failure')
    x.append(result_fcc.x[2])
    for compset in fcc_composition_sets:
        y[compset.phase_record.phase_name].append(float(compset.energy))
    for compset in bcc_composition_sets:
        y[compset.phase_record.phase_name].append(float(compset.energy))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycalphad.plot.utils import phase_legend
stable_phases = ['FCC_A1', 'BCC_A2']

energy_diff = np.array(y['FCC_A1']) - np.array(y['BCC_A2'])
sign_change_idx = np.where(np.diff(np.sign(energy_diff)) != 0)[0] + 1
sign_change_temps = np.array(x)[sign_change_idx]

plt.plot(x, energy_diff)
plt.scatter(sign_change_temps, np.zeros_like(sign_change_temps), c='r', zorder=3)
plt.ylabel('Gibbs Energy Difference (J/mol-atom)')
plt.xlabel('Temperature (K)')
plt.xlim(300, 2000)

In [ ]:
print('FCC->BCC T0 Temperature: ', sign_change_temps[0], 'K', f'({sign_change_temps[0]-273.15} deg C)')

# Precipitation Reaction
Note that the temperature corresponding to the onset of the corresponding precipitation reaction will always be greater than (or equal to) the T0 temperature. However, we may not always observe the precipitation reaction due to the slower, diffusion-controlled kinetics of the reaction. We can compute the onset of bcc stability by 'fixing' the bcc phase to be stable with zero amount.

In [ ]:
comps = ['FE', 'MN', 'VA']

state_variables = {v.N: 1, v.P: 1e5}

composition_sets = [
                    composition_set(dbf, comps, 'BCC_A2',
                                    {**state_variables, v.T: 300},
                                    fixed=True, phase_amt=0,
                                   ),
                    composition_set(dbf, comps, 'FCC_A1',
                                    {**state_variables, v.T: 300},
                                    fixed=False, phase_amt=1,
                                   ),  
                   ]

result, composition_sets = local_equilibrium(composition_sets, comps, {**state_variables, v.X('MN'): 0.1})
print('Converged: ', result.converged)
print('Final Composition Sets: ', composition_sets)
print('FCC-BCC Transus Temperature', result.x[2], 'K', f'({int(result.x[2]-273.15)} deg C)')